In [ ]:
import pandas as pd
import requests

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      parse_gnomad,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      set_gnomad_dtypes,
                      request_gnomad_api_data,
                      merge_gnomad_lovd,
                      GNOMAD_PATH,
                      )
from api.data import save_lovd_as_vcf


pd.options.display.max_columns = 0

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [ ]:
gnomad_data = request_gnomad_api_data("EYS")

display(gnomad_data)

In [ ]:
store_database_for_eys_gene('gnomad', False)

gnomad_data_2 = parse_gnomad(GNOMAD_PATH +'/gnomad_data.csv')

In [ ]:
display(gnomad_data_2)

In [ ]:
gnomad_data_2.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_downloaded.csv', index=False)
gnomad_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_api.csv', index=False)

In [ ]:
len(gnomad_data_2), len(gnomad_data)

print(len(gnomad_data_2) - len(gnomad_data))

In [ ]:
gnomad_data

In [ ]:
missing_from_api = []

for i in gnomad_data['gnomAD ID']:
    if(i in gnomad_data_2['gnomAD ID'].values):
        continue
    missing_from_api.append(i)

len(missing_from_api)

missing_data = gnomad_data.loc[gnomad_data['gnomAD ID'].isin(missing_from_api)]

missing_data

In [ ]:
missing_data.to_csv('C:\\Users\\Kajus\\Desktop\\gnomad_data_missing.csv', index=False)

In [ ]:
set_lovd_dtypes(data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = data["Variants_On_Genome"].copy()

lovd_data = pd.merge(data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data

In [ ]:
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

In [ ]:
from subprocess import Popen

process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)

In [ ]:
from api.data.refactoring import merge_gnomad_lovd, parse_gnomad, set_gnomad_dtypes
import pandas as pd
from api import (store_database_for_eys_gene,
                 parse_lovd,
                 set_lovd_dtypes,
                 LOVD_PATH,
                 GNOMAD_PATH)

store_database_for_eys_gene('lovd', False)
store_database_for_eys_gene('gnomad', False)

lovd_data = parse_lovd(LOVD_PATH + "/lovd_data.txt")
gnomad_data = parse_gnomad(GNOMAD_PATH+'/gnomad_data.csv')

set_lovd_dtypes(lovd_data)
set_gnomad_dtypes(gnomad_data)

variants_on_genome = lovd_data["Variants_On_Genome"].copy()

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id','VariantOnGenome/DNA','VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
final_data = merge_gnomad_lovd(lovd_data, gnomad_data)
final_data